In [ ]:
import json

import pandas as pd
import geopandas as gpd

from shapely.geometry import shape, mapping

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from shapely.geometry import 

### Load processed UDL data

In [ ]:
# raw contour data
with open('./data/contours_20220505.json') as F: t_contours = json.load(F)

# flatten the data
flat = []
for V in t_contours.values(): flat.extend( V )
t_contours_df = pd.DataFrame( flat )

# make geometries, and set the beamId as the index
shapes = t_contours_df['geographyJson'].apply( lambda X: shape(X) )
t_contours_df[ 'geometry' ] = gpd.GeoSeries( shapes )
t_contours_df[ 'myidx' ] = t_contours_df[ 'idBeam'].copy()
t_contours_df.set_index( 'myidx', inplace=True )

# UDL channels, beam, RF data (match up with t_contours via idBeam)
t_udl = pd.read_json('./derived_data/udl_merged_no_contours.json')

### Look for uplink beams

In [ ]:
uplink = t_udl[ t_udl['mode'].str.contains('RX' )]

In [ ]:
def getLLA( V ):
    tv = mapping( V )
#     assert 1 == 2
    if str(V.type) == 'Point': 
        return [{'lon' : tv['coordinates'][0], 'lat' : tv['coordinates'][1]}]
    
    if str(V.type) == 'LineString': 
        return [ {'lon' : X[0], 'lat' : X[1] } for X in tv['coordinates'] ]
    
    if str(V.type) == 'Polygon' :
        return [ {'lon' : X[0], 'lat' : X[1] } for X in tv['coordinates'][0] ]

             

test = t_contours_df['geometry'].apply( getLLA )


In [ ]:
test.iloc[0]

In [ ]:
t_contours_df

In [ ]:
for i,R in t_contours_df.iterrows():
    print( mapping(R['geometry']))

In [ ]:
# make a list grouped by NORAD...

for NOR, G in uplink.groupby('NORAD'):
    newG = G.copy()
    newG.sort_values( by='freqMin', inplace=True )
    for i,R in newG.iterrows():
        print('{:20} {:10.3f} {:10.3f} {:10.3f} {} {}'.format( R['idTransponder'], R['freqMin'], R['freqMax'], R['centerFreq'], R['idBeam'], R['polarization']))
    print()

### Overlaps

- look for situations where the beam, freq, and transponder seem to overlap

In [ ]:
collisions = []
for col, G in uplink.groupby( ['NORAD','freqMin','freqMax','centerFreq','idBeam','polarization'] ):
    if len(G) > 1: collisions.append( G.copy () )

In [ ]:
for c in collisions:
    if len(c) > 30:
        trans = c.iloc[0]['idTransponder']
        print(trans)
        print(c['idBeam'])

In [ ]:
for col in collisions[10].columns:
    print( collisions[10][col])

In [ ]:
collis[1].columns

In [ ]:
gpd.GeoDataFrame( t_contours_df[ t_contours_df['idBeam'] == '7cb8be66-fc4d-4ec7-9de6-2b53741e22ce']).plot()

In [ ]:
gpd.GeoDataFrame( t_contours_df[ t_contours_df['idBeam'] == '3f9d5c2a-231f-4281-bd43-57b088275511']).plot()

In [ ]:
c

In [ ]:
# T[ T['geographyType'].str.contains( 'LineString' ) ].plot()
for G in uplink['geometry'] : 
    mapping(G)
    
#     print( mapping(G))
    print(G)
    print(dir(G))
    print(str(G.type))
#     print(len(mapping(G)['coordinates']))
#     print(mapping(G)['coordinates'])
    print()
# mapping( T.iloc[1]['geometry'])

In [ ]:
G.